In [3]:
import pandas as pd
import scipy.io
import glob
import numpy as np 
import mne 

In [6]:
# read the excel files just in case 
patients_data = pd.read_excel("3-June2022/Alle_patienten_250Hz.xlsx")
patients_data1 = pd.read_excel("3-JuneAlle_patienten_256Hz.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: '3-June2022/Alle_patienten_250Hz.xlsx'

In [ ]:
patients_data = patients_data.rename(columns = {"Burstsupressie (ja/nee)" : "BS",
                                                "Nummer" : "ID"})
patients_data = patients_data.dropna(subset = ["ID"])
patients_data = patients_data.astype({"ID" : int})
patients_data

: 

In [ ]:
patients_data1 = patients_data1.rename(columns = {"Burstsupressie (ja/nee)" : "BS",
                                                "Nummer" : "ID"})
patients_df = pd.concat([patients_data, patients_data1])
patients_df = patients_df.drop(["file code"], axis = 1)

: 

In [ ]:
not_annoted = patients_df[patients_df["BS"].isna()]
not_annoted

: 

In [ ]:
annotated_patients = patients_df[patients_df['BS'].notna()]
annotated_patients

: 

In [ ]:
patients = glob.glob("3-June2022/*.mat")
len(patients)

: 

In [ ]:
# necessary information taken from the data in order to create MNE objects. 
channelLabels = ["Fp1", "Fp2", "F7", "F3", "Fz", "F4", "F8",
                "A1", "T3", "C3", "Cz", "C4", "T4", "A2", "T5", 
                "P3", "Pz", "P4", 'T6', '01', '02', 'ECG']
trials = 1 
n_channels = 22
sampling_frequency = 250
ch_types = ['eeg'] * (n_channels-3)
ch_types.append('misc')
ch_types.append('misc')
ch_types.append('ecg')

: 

In [ ]:
def define_montage(type):
    '''method that helps us define the channel locations '''

    # Form the 10-20 montage 'standard_1020'
    mont1020 = mne.channels.make_standard_montage(type)
    # Choose what channels you want to keep 
    # Make sure that these channels exist e.g. T1 does not exist in the standard 10-20 EEG system!
    kept_channels = ["Fp1", "Fp2", "F7", "F3", "Fz", "F4", "F8",
                "A1", "T3", "C3", "Cz", "C4", "T4", "A2", "T5", 
                "P3", "Pz", "P4", 'T6']
    ind = [i for (i, channel) in enumerate(mont1020.ch_names) if channel in kept_channels]
    mont1020_new = mont1020.copy()
    # Keep only the desired channels
    mont1020_new.ch_names = [mont1020.ch_names[x] for x in ind]
    kept_channel_info = [mont1020.dig[x+3] for x in ind]
    # Keep the first three rows as they are the fiducial points information
    mont1020_new.dig = mont1020.dig[0:3]+kept_channel_info
    
    #mont1020.plot()
    #mont1020_new.plot()
    
    return mont1020_new



: 

In [ ]:
def create_MNE_objects(patient_list,channelLabels,n_channels,sampling_frequency,phase,types): 
    evoked_arrays = []
    for i in range(0,len(patient_list)): 
        patient = scipy.io.loadmat(patient_list[i])
        samples = patient[phase]["data"][0][0]
        
        info = mne.create_info(
        ch_names = channelLabels, 
        ch_types = types,
        sfreq = sampling_frequency)
        
        info.set_montage(define_montage('standard_1020'))
        evoked_array = mne.EvokedArray(samples, info,
                               nave=samples.shape[0], comment=('patient' + str(i)))
        evoked_arrays.append(evoked_array)
    return evoked_arrays

: 

In [ ]:
eeg_InDia = create_MNE_objects(patients, channelLabels, n_channels, sampling_frequency,"EEG_INDIA",ch_types)
eeg_PreOK = create_MNE_objects(patients, channelLabels, n_channels, sampling_frequency, "EEG_PreOK",ch_types)
eeg_Klem = create_MNE_objects(patients, channelLabels, n_channels, sampling_frequency, "EEG_Klem",ch_types)

: 

In [ ]:
#for i in range (5,10): 
 #   eeg_InDia[i].plot(spatial_colors=True)

: 